In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

wrsamp calls wrheader with sig. wrheader calls setfields, and checkfields. 


setfields() - fills in fields of a wfdbrecord object. Fields that it can fill in include essential fields, and dependencies.
It will NOT overwrite any user defined fields, aka fields != None. 

In [12]:
class WFDBrecord():
    
    # Constructor
    def __init__(self, signals=None, recordname=None, nsig=None, 
                 fs=None, counterfreq=None, basecounter=None, 
                 siglen=None, basetime=None, basedate=None, 
                 filename=None, fmt=None, sampsperframe=None, 
                 skew=None, byteoffset=None, adcgain=None, 
                 baseline=None, units=None, adcres=None, 
                 adczero=None, initvalue=None, checksum=None, 
                 blocksize=None, signame=None, comments=None):
        
        # Note the lack of 'nseg' field. Single segment records cannot have this field. Even nseg = 1 makes 
        # the header a multi-segment header. 
        
        self.signals = signals
        
        self.recordname=recordname
        self.nsig=nsig
        self.fs=fs
        self.counterfreq=counterfreq
        self.basecounter=basecounter
        self.siglen=siglen
        self.basetime=basetime
        self.basedate=basedate
        
        self.filename=filename
        self.fmt=fmt
        self.sampsperframe=sampsperframe
        self.skew=skew
        self.byteoffset=byteoffset
        self.adcgain=adcgain
        self.baseline=baseline
        self.units=units
        self.adcres=adcres
        self.adczero=adczero
        self.initvalue=initvalue
        self.checksum=checksum
        self.blocksize=blocksize
        self.signame=signame
        
        self.comments=comments

In [35]:
import numpy as np
# The specifications of a WFDB field
# 0. type
# 1. delimiter
# 2. dependency
# 3. required to write? (nseg starts as F but will be set by wrmultiheader and wrmultiseg to True.)
# 4. default fill function available? 0-2. 
class WFDBfieldspecs():
    
    def __init__(self, speclist):
    
        # Data types the field can take
        self.allowedtypes = speclist[0]
        
        # The text delimiter that preceeds the field
        self.delimiter = speclist[1]
        
        # The required/dependent field which must also be present
        self.dependency = speclist[2]
        
        # Whether the field is mandatory for writing a header (WFDB requirements + extra rules enforced by this library).
        # Being required for writing is not the same as the user having to specify via wrsamp/wrhea. These functions can set defaults.
        self.write_req = speclist[3]
        
        # Whether there is a default value for this field that can be inferred or calculated, and also whether ther setfield function will actually set the field. 
        # 0 = no, 1 = yes but only if signals is present, 2 = yes without needing signals (although it may still use it).  
        # Watch out for: adcgain, baseline. Set them to 1 but have a check statement in the function. Should be 0 if isdigital.
        # sampsperframe, skew, and byteoffset do have defaults but the setfields will not return anything because we do not want to write anything. So value = 0. Conversely when reading, these fields will be left as None if not present. 
        # This field is used to see if we can call the 'setfield' function on that field.
        self.has_write_default = speclist[4]
         

# The signal field
signalspecs = OrderedDict([('signal', WFDBfieldspecs([[np.ndarray], '', None, False, 2]))])

# The segment field
segmentspecs = OrderedDict([('segment', WFDBfieldspecs([[list], '', None, True, 2]))])

# Record specification fields            
recfieldspecs = OrderedDict([('recordname', WFDBfieldspecs([[str], '', None, True, 0])),
                         ('nseg', WFDBfieldspecs([[int], '/', 'recordname', True, 0])), # Essential for multi but illegal for single.
                         ('nsig', WFDBfieldspecs([[int], ' ', 'recordname', True, 1])),
                         ('fs', WFDBfieldspecs([[int, float], ' ', 'nsig', True, 0])),
                         ('counterfreq', WFDBfieldspecs([[int, float], '/', 'fs', False, 0])),
                         ('basecounter', WFDBfieldspecs([[int, float], '(', 'counterfreq', False, 0])),
                         ('siglen', WFDBfieldspecs([[int], ' ', 'fs', True, 1])),
                         ('basetime', WFDBfieldspecs([[str], ' ', 'siglen', False, 2])),
                         ('basedate', WFDBfieldspecs([[str], ' ', 'basetime', False, 0]))])
# Signal specification fields 
sigfieldspecs = OrderedDict([('filename', WFDBfieldspecs([[str], '', None, True, 2])),
                         ('fmt', WFDBfieldspecs([[int, str], ' ', 'filename', True, 2])),
                         ('sampsperframe', WFDBfieldspecs([[int], 'x', 'fmt', False, 0])),
                         ('skew', WFDBfieldspecs([[int], ':', 'fmt', False, 0])),
                         ('byteoffset', WFDBfieldspecs([[int], '+', 'fmt', False, 0])),
                         ('adcgain', WFDBfieldspecs([[int], ' ', 'fmt', True, 1])),
                         ('baseline', WFDBfieldspecs([[int], '(', 'adcgain', True, 1])),
                         ('units', WFDBfieldspecs([[str], '/', 'adcgain', True, 2])),
                         ('adcres', WFDBfieldspecs([[int], ' ', 'adcgain', False, 2])),
                         ('adczero', WFDBfieldspecs([[int], ' ', 'adcres', False, 2])),
                         ('initvalue', WFDBfieldspecs([[int], ' ', 'adczero', False, 1])),
                         ('checksum', WFDBfieldspecs([[int], ' ', 'initvalue', False, 1])),
                         ('blocksize', WFDBfieldspecs([[int], ' ', 'checksum', False, 2])),
                         ('signame', WFDBfieldspecs([[str], ' ', 'blocksize', False, 1]))])
    
# Segment specification fields
segfieldspecs = OrderedDict([('segname', WFDBfieldspecs([[str], '', None, True, 0])),
                         ('seglen', WFDBfieldspecs([[int], ' ', 'segname', True, 0]))])
# Comment field
comfieldspecs = OrderedDict([('comments', WFDBfieldspecs([[int], '', None, False, False]))])


singlefieldspeclist = [signalspecs.copy(), recfieldspecs.copy(), sigfieldspecs.copy(), comfieldspecs.copy()]
del(singlefieldspeclist[1]['nseg'])
multifieldspeclist = [segmentspecs.copy(), recfieldspecs.copy(), segfieldspecs.copy(), comfieldspecs.copy()]

In [52]:
fieldspeclist = singlefieldspeclist
for f in OrderedDict(reversed(list(fieldspeclist[1].items()))):
    print(f)

basedate
basetime
siglen
basecounter
counterfreq
fs
nsig
recordname


In [56]:
a=[1,2,3]
b=a
print(a is b)
b=[2,3,4]
a is b

True


False

In [49]:
a=[1,2,3]
b=a.copy()
b is a

False

In [30]:
rev_rfields=OrderedDict(reversed(list(WFDBfieldlist[0].items())))

In [32]:
rev_rfields['basedate'].write_req

False

In [51]:
a=None
if a and a>5:
    print('hi')

TypeError: unorderable types: NoneType() > int()

In [1]:
from testmodule import bread

In [4]:
panini=bread()
panini.getcals(3)

300

In [3]:
dir(panini)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'carbs',
 'getcals']

In [6]:
from collections import OrderedDict

a=OrderedDict([('name','jack'),('age',50)])



In [7]:
a

OrderedDict([('name', 'jack'), ('age', 50)])

In [11]:
del(a['name'])
a

OrderedDict([('age', 50)])

In [57]:
a=np.array([1,2,3])
b=list(a)
b

[1, 2, 3]

In [12]:
class dumbclass():
    
    def something(self, x):
        return x*2

In [13]:
a=dumbclass()
b=a.something(5)

In [14]:
b

10

In [15]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'something']

In [1]:
import wfdb